In [22]:
import tensorflow as tf
from tensorflow.python.platform import gfile
import tensorflow as tf
import os
import sys
from tensorflow.python.platform import gfile
import numpy as np
from PIL import Image
import lime
import lime.lime_tabular 
from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet
from keras import applications
import glob
import matplotlib.pyplot as plt

In [3]:
GRAPH_PB_PATH = 'C:\\Users\\Hp\\Downloads\\satellite-image-deep-learning-master\\satellite-image-deep-learning-master\\land_classification\\tensorflow\\mobilenet_v2\\output_graph.pb'
with tf.Session() as sess:
   print("load graph")
   with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
       graph_def = tf.GraphDef()
   graph_def.ParseFromString(f.read())
   sess.graph.as_default()
   tf.import_graph_def(graph_def, name='')
   graph_nodes=[n for n in graph_def.node]
   names = []
   for t in graph_nodes:
      names.append(t.name)
   print(names)

load graph
Instructions for updating:
Use tf.gfile.GFile.
['Placeholder', 'module/MobilenetV2/Conv/weights', 'module/MobilenetV2/Conv/BatchNorm/gamma', 'module/MobilenetV2/Conv/BatchNorm/beta', 'module/MobilenetV2/Conv/BatchNorm/moving_mean', 'module/MobilenetV2/Conv/BatchNorm/moving_variance', 'module/MobilenetV2/expanded_conv/depthwise/depthwise_weights', 'module/MobilenetV2/expanded_conv/depthwise/BatchNorm/gamma', 'module/MobilenetV2/expanded_conv/depthwise/BatchNorm/beta', 'module/MobilenetV2/expanded_conv/depthwise/BatchNorm/moving_mean', 'module/MobilenetV2/expanded_conv/depthwise/BatchNorm/moving_variance', 'module/MobilenetV2/expanded_conv/project/weights', 'module/MobilenetV2/expanded_conv/project/BatchNorm/gamma', 'module/MobilenetV2/expanded_conv/project/BatchNorm/beta', 'module/MobilenetV2/expanded_conv/project/BatchNorm/moving_mean', 'module/MobilenetV2/expanded_conv/project/BatchNorm/moving_variance', 'module/MobilenetV2/expanded_conv_1/expand/weights', 'module/Mobilenet

In [4]:
model_path = "./mobilenet_v2/output_graph.pb"
file_size = int(os.stat(model_path).st_size/1e6)
print("The model (`.pb`) file {} MB".format(file_size))

The model (`.pb`) file 9 MB


In [5]:
label_file = "./mobilenet_v2/output_labels.txt"

In [6]:
from label_image import load_graph, load_labels, read_tensor_from_image_file

In [7]:
global graph
graph = load_graph(model_path)

In [18]:
global img_path
img_path = 'C:\\Users\\Hp\\Downloads\\6038468-637595f25e2bfe3f004b7761e94db1d8b2d6c77e\\6038468-637595f25e2bfe3f004b7761e94db1d8b2d6c77e\\f4.PNG'

In [9]:
input_operation = graph.get_operation_by_name("import/Placeholder")
output_operation = graph.get_operation_by_name("import/final_result")

input_height = 224
input_width = 224
images_list = []
# for i in range(1,11):
# img_path_new = img_path + str(i) + '.PNG'
global t
t = read_tensor_from_image_file(img_path, input_height=input_height, input_width=input_width) 

print(t.shape)
    
#     with tf.Session(graph=graph) as sess:
#         results = sess.run(output_operation.outputs[0], {
#             input_operation.outputs[0]: t
#         })
#     results = np.squeeze(results)
#     top_k = results.argsort()[-5:][::-1]
#     labels = load_labels(label_file)
#     print("Results for image {} \n".format(i)

#     for j in top_k:
#         print(labels[j], results[j])
        
#     print("\n\n")
    
#     print(t.shape)
#     list.append(t)
    


(1, 224, 224, 3)


In [10]:
with tf.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
results = np.squeeze(results)

top_k = results.argsort()[-5:][::-1]
labels = load_labels(label_file)

for i in top_k:
    print(labels[i], results[i])
    
print(results)

oil gas field 0.3096601
cemetery 0.24338113
beach 0.169541
ferry terminal 0.06387028
river 0.05608331
[1.9025896e-03 5.5156965e-03 1.5529247e-02 1.6954100e-01 1.5762953e-02
 2.4338113e-01 1.1615172e-02 9.2418706e-03 1.8725684e-03 9.3947056e-05
 1.6723844e-04 1.0318784e-04 6.3870281e-02 2.2277522e-03 2.0352665e-02
 1.1024425e-03 5.0600348e-03 1.1285259e-03 3.8018383e-03 1.1476145e-04
 1.4335882e-03 3.0966011e-01 8.6848224e-03 2.5477579e-03 3.4399787e-03
 1.5228194e-03 1.1969114e-03 5.6083310e-02 5.3739161e-03 4.9087079e-03
 5.2393065e-04 6.4022066e-03 3.6014791e-04 2.9069185e-03 1.4055961e-04
 8.9363279e-03 1.0666625e-02 2.8264492e-03]


In [11]:
predict_model = lambda x: model.predict_proba(x).astype(float)

feature_list = list(i.strip("\n") for i in open("./mobilenet_v2/output_labels.txt").readlines())
print(feature_list)

print(len(feature_list))

['airplane', 'baseball field', 'basketball court', 'beach', 'bridge', 'cemetery', 'chaparral', 'christmas tree farm', 'closed road', 'coastal mansion', 'crosswalk', 'dense residential', 'ferry terminal', 'football field', 'forest', 'freeway', 'golf course', 'harbor', 'intersection', 'mobile home park', 'nursing home', 'oil gas field', 'oil well', 'overpass', 'parking lot', 'parking space', 'railway', 'river', 'runway', 'runway marking', 'shipping yard', 'solar panel', 'sparse residential', 'storage tank', 'swimming pool', 'tennis court', 'transformer station', 'wastewater treatment plant']
38


In [13]:
explainer = lime_image.LimeImageExplainer()

In [20]:
%load_ext autoreload
%autoreload 2
import os,sys
import glob
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
img_path = glob.glob(img_path, recursive=True)[0]
# print(img_path)
img = plt.imread(img_path)
# print(img.shape)
# plt.imshow(img);

global data
data = np.asarray(img)
# print(data)

In [32]:
# print(len(t.shape))
# print(data)
explanation = explainer.explain_instance(t, results, top_labels=5, hide_color=0, num_samples=1000)

[[[0.5176471  0.42745098 0.3882353  1.        ]
  [0.5176471  0.42745098 0.3882353  1.        ]
  [0.5176471  0.4117647  0.3882353  1.        ]
  ...
  [0.48235294 0.4117647  0.3882353  1.        ]
  [0.5019608  0.42352942 0.40392157 1.        ]
  [0.45490196 0.38039216 0.35686275 1.        ]]

 [[0.5176471  0.42745098 0.3882353  1.        ]
  [0.5176471  0.41960785 0.3882353  1.        ]
  [0.5176471  0.4117647  0.3882353  1.        ]
  ...
  [0.48235294 0.4117647  0.3882353  1.        ]
  [0.5137255  0.43137255 0.41568628 1.        ]
  [0.47843137 0.40392157 0.38431373 1.        ]]

 [[0.5176471  0.41568628 0.3882353  1.        ]
  [0.5058824  0.4117647  0.3882353  1.        ]
  [0.49411765 0.4117647  0.3882353  1.        ]
  ...
  [0.48235294 0.39607844 0.36078432 1.        ]
  [0.49411765 0.41960785 0.4        1.        ]
  [0.4862745  0.4117647  0.39607844 1.        ]]

 ...

 [[0.37254903 0.3647059  0.28627452 1.        ]
  [0.34901962 0.3372549  0.25882354 1.        ]
  [0.32549

RuntimeError: sequence argument must have length equal to input rank